<h1>Scraping Fidelity.com</h1>
In this assignment, you will scrape data from fidelity.com. The goal of the exercise is to get the latest sector performance data from the US markets, and to get the total market capitalization for each sector. 

The end result is to write a function: <i>get_us_sector_performance()</i> that will return a list of tuples. Each tuple should correspond to a sector and should contain the following data:
<li>the sector name
<li>the amount the sector has moved
<li>the market capitalization of the sector
<li>the market weight of the sector
<li>a link to the fidelity page for that sector

<p>
The data should be sorted by decreasing order of market weight. I.e., the sector with the highest weight should be in the first tuple, etc.

<h3>Process</h3>
<li>Get a list of sectors and the links to the sector detail pages from the url (see function)
<li>Loop through the list and call the function <i>get_sector_change_and_market_cap(sector_page_link)</i> for each sector
<li>Accumulate the name, the change, the capitalization, the weight and the link for each sector in output_list (see function)
<li>Sort the list by market weight

<b>Notes:</b>
<li>Note that the market weight is a string with a % sign at the back. You will need to get rid of the % and convert the string into a float before you can sort it
<li>Your starting data is the url listed below. You need to extract all data, including links to the sector pages, from the page at this url
<li>To sort a list of tuples by an arbitrary element, use the example at the bottom of this notebook

In [2]:
def get_us_sector_performance():
    output_list = list()
    output_tup = ()
    import requests
    from bs4 import BeautifulSoup
    url = "https://eresearch.fidelity.com/eresearch/goto/markets_sectors/landing.jhtml"
    response = requests.get(url)
    if not response.status_code == 200:
        return None
    try:
        results_page = BeautifulSoup(response.content,'lxml')
        sectors = results_page.find_all('tr')
        for sector in sectors:
            try:
                sector_name = sector.find('strong').get_text()
                sector_link = "https://eresearch.fidelity.com/" + sector.find('a',class_="heading1").get('href')
                amnt_moved = get_sector_change_and_market_cap(sector_link)[0]
                mrkt_cap = get_sector_change_and_market_cap(sector_link)[1]
                mrkt_weight = get_sector_change_and_market_cap(sector_link)[2]
            except:
                sector_name = ''
            if sector_name != '':
                output_tup = sector_name +sector_link
                output_list.append((sector_name,amnt_moved,mrkt_cap,mrkt_weight,sector_link))
        output_list.sort(key=lambda k: k[3],reverse=True)
        return output_list
    except:
        return None

In [1]:
def get_sector_change_and_market_cap(sector_page_link):
    span_vars = list()
    import requests
    from bs4 import BeautifulSoup
    url = sector_page_link
    response = requests.get(url)
    if not response.status_code == 200:
        return None
    try:
        results_page = BeautifulSoup(response.content,'lxml')
        sectors = results_page.find_all('td')
        count = 0
        for sector in sectors:
            span_vars.append(sector.find('span').get_text())
            count += 1
            if count > 2:
                break
    except:
        return None
    
    if span_vars[0][0] == '+':
        sector_change = float(span_vars[0][1:-1])
    else:
        sector_change = float(span_vars[0][:-1])
    sector_market_cap = span_vars[1]
    sector_market_weight = float(span_vars[2][:-1])
    
    return sector_change,sector_market_cap,sector_market_weight

In [3]:
#Test get_sector_change_and_market_cap()
link = "https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=25"
get_sector_change_and_market_cap(link)
#Should return (-0.40, $3.58T, 6.80)
#Note that neither the -0.40, nor the 6.80, end with a %sign

/opt/anaconda3/lib/python3.8/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


(-0.66, '$9.35T', 12.51)

In [4]:
#Test get_us_sector_performance()
get_us_sector_performance()
#Should return (example: obviously the results will vary over time!)
"""
[('Telecommunication Services',
  0.21,
  '$1.74T',
  2.0,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=50'),
 ('Materials',
  0.22,
  '$1.95T',
  2.49,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=15'),
 ('Real Estate',
  -0.45,
  '$1.22T',
  2.7,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=60'),
 ('Utilities',
  -0.33,
  '$1.25T',
  2.86,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=55'),
 ('Energy',
  0.76,
  '$3.90T',
  5.83,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=10'),
 ('Consumer Staples',
  -0.32,
  '$3.58T',
  6.8,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=30'),
 ('Industrials',
  0.72,
  '$4.31T',
  9.83,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=20'),
 ('Consumer Discretionary',
  1.03,
  '$5.76T',
  12.9,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=25'),
 ('Financials',
  0.39,
  '$7.45T',
  13.71,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=40'),
 ('Health Care',
  0.76,
  '$5.70T',
  14.71,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=35'),
 ('Information Technology',
  0.81,
  '$9.84T',
  26.17,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=45')]
  
"""

"\n[('Telecommunication Services',\n  0.21,\n  '$1.74T',\n  2.0,\n  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=50'),\n ('Materials',\n  0.22,\n  '$1.95T',\n  2.49,\n  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=15'),\n ('Real Estate',\n  -0.45,\n  '$1.22T',\n  2.7,\n  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=60'),\n ('Utilities',\n  -0.33,\n  '$1.25T',\n  2.86,\n  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=55'),\n ('Energy',\n  0.76,\n  '$3.90T',\n  5.83,\n  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=10'),\n ('Consumer Staples',\n  -0.32,\n  '$3.58T',\n  6.8,\n  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=30'

<h3>Sorting</h3>

In [ ]:
x = [('a',23.2,'b'),('c',17.4,'f'),('d',29.2,'z'),('e',1.74,'bb')]
x.sort() #Sorts by the first element of the tuple
x

In [ ]:
x = [('a',23.2,'b'),('c',17.4,'f'),('d',29.2,'z'),('e',1.74,'bb')]
x.sort(key=lambda k: k[1]) #Sorts by the element at position 1
x